In [1]:
# Core
import os
import pickle

# Data manipulation & visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
from PIL import Image
import cv2

# Machine learning / Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import DenseNet121, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
)

# Scikit-learn
from sklearn.model_selection import train_test_split


In [3]:
# load facial key points data
keyfacial_df = pd.read_csv(r'data.csv')

In [ ]:
keyfacial_df